In [1]:

from __future__ import print_function
import json
import time
import datetime
import pandas as pd

# Python 2 and 3: alternative 4
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"


def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return stations


def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    states = "RS"
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ["ASOS"]
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = (
            "https://mesonet.agron.iastate.edu/" "geojson/network/%s.geojson"
        ) % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict["features"]:
            stations.append(site["properties"]["sid"])
    return stations

def main():
    """Our main method"""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2024, 1, 1)
    endts = datetime.datetime.now()

    #service = SERVICE + "data=mslp&=Etc/UTC&format=comma&=yes&"

    #service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
    #service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

    # Two examples of how to specify a list of stations
    #stations = get_stations_from_networks()
    stations = ['LYBE']
    print(stations)
    print(type(stations))
    # stations = get_stations_from_filelist("mystations.txt")
    #print(service)
    
    dfs = []
    
    data_types = ["alti", "tmpf"]
    for station in stations:
        
        for data_type in data_types:
            
            service = SERVICE + "data="+data_type+"&=Etc/UTC&format=onlytdf&=yes&"

            service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
            service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

            uri = "%s&station=%s" % (service, station)
            print("Downloading: %s - %s" % (station, data_type))
            data = download_data(uri)
            outfn = "%s_%s_%s.txt" % (
                station,
                startts.strftime("%Y%m%d%H%M"),
                endts.strftime("%Y%m%d%H%M"),
            )
            out = open(outfn, "w")
            out.write(data)
            
            df = pd.read_csv(outfn, sep='\t' )
            out.close()
            dfs.append(df)
        
            df.head()
            print(df)
    
    final_dataframe = pd.merge(dfs[0], dfs[1])
    print(final_dataframe)
    final_dataframe.to_csv("mslp_temp_Belgrade.csv",index = False)

if __name__ == "__main__":
    main()

['LYBE']
<class 'list'>
Downloading: LYBE - alti
     station             valid   alti
0       LYBE  2024-01-01 00:00  29.94
1       LYBE  2024-01-01 00:30  29.91
2       LYBE  2024-01-01 01:00  29.91
3       LYBE  2024-01-01 01:30  29.91
4       LYBE  2024-01-01 02:00  29.91
...      ...               ...    ...
9157    LYBE  2024-07-09 21:30  29.97
9158    LYBE  2024-07-09 22:00  29.97
9159    LYBE  2024-07-09 22:30  29.97
9160    LYBE  2024-07-09 23:00  29.97
9161    LYBE  2024-07-09 23:30  29.97

[9162 rows x 3 columns]
Downloading: LYBE - tmpf
     station             valid  tmpf
0       LYBE  2024-01-01 00:00  44.6
1       LYBE  2024-01-01 00:30  44.6
2       LYBE  2024-01-01 01:00  42.8
3       LYBE  2024-01-01 01:30  41.0
4       LYBE  2024-01-01 02:00  44.6
...      ...               ...   ...
9157    LYBE  2024-07-09 21:30  82.4
9158    LYBE  2024-07-09 22:00  82.4
9159    LYBE  2024-07-09 22:30  80.6
9160    LYBE  2024-07-09 23:00  82.4
9161    LYBE  2024-07-09 23:30  80.6

